<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Exploración Datalake
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>


## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Abril 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Consideraciones para la consulta en datos abiertos**|
|**Descripción y alcance**|Script para la consulta de bases y exploración de bases en el DATAVAULT.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC<td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

![image.png](../../../muestras%20de%20datos/sin%20procesar/descarga_bodega_.png)

## Instalaciones previas
 
Para poder utilizar SQL en Python es necesario instalar lo siguiente, escoja el driver de acuerdo al sistema operativo de su equipo: 
 
 
**Driver Microsoft ODBC para SQL SERVER en windows**
 
Descargar e instalar el driver de la página [Microsoft ODBC](https://docs.microsoft.com/en-us/sql/connect/odbc/windows/system-requirements-installation-and-driver-files?view=sql-server-ver15#installing-microsoft-odbc-driver-for-sql-server) versión 
 17.
 
**Driver Microsoft ODBC para SQL SERVER en Linux o MacOs**
 
Descargar e instalar el driver de la página [Microsoft ODBC](https://docs.microsoft.com/en-us/sql/connect/odbc/linux-mac/system-requirements?view=sql-server-ver15) versión 17 deacuerdo a la versión de su sistema operativo.
Recuerde estar conectado a la VPN para poder accerder. 

**Paquete pyobdc**

Para instalar el paquete pyodbc ejecute el siguiente comando en la consola de su equipo:

```python
pip install pyodbc
```

**Paquete pandas**

Se sugiere una versión mayor que la de 2.0.1 si no tiene instalado pandas ejecute el siguiente comando en la consola de su equipo:

```python
pip install pandas
```

Si lo tiene pero desactualizado ejecute el siguiente comando en la consola de su equipo:

```python
pip install --upgrade pandas
```

Para este ejercicio también es necesario instalar el paquete pyarrow, para ello ejecute el siguiente comando en la consola de su equipo:

```python
pip install pyarrow
```



In [ ]:
import pyodbc 
from dotenv import load_dotenv
import os
load_dotenv()

server_reports= os.getenv("server_reports")
server_sandbox= os.getenv("server_sandbox")
username= os.getenv("username")
password= os.getenv("password")

cnxn_reports = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server_reports+';UID='+username+';PWD='+ password)
cursor_reports = cnxn_reports.cursor()

cnxn_sandbox = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server_sandbox+';UID='+username+';PWD='+ password)
cursor_sandbox = cnxn_sandbox.cursor()


ruta_DATA="../../../../DATA/"
ruta_MUESTRAS="../../../muestras de datos/sin procesar/MUESTRAS/ "

In [ ]:
### Paquetes usados para la exploración de datos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
from scripts import data_management as dm # Funciones definidas
warnings.filterwarnings('ignore')

## 3. Tablas Requeridas

En particular de aqui saldran las siguientes tablas para poner a disposición del datalake de desarrollo del equipo de datos de EMAE:



- **Tabla de procesos SECOP II**: Contiene la información de los procesos que se han llevado a cabo en el Sistema Electrónico de Contratación Pública SECOP II.

- **Tabla de contratos SECOP II**: Contiene la información de los contratos que se han llevado a cabo en el Sistema Electrónico de Contratación Pública SECOP II.


- **Tabla de procesos SECOP I**: Contiene la información de los procesos que se han llevado a cabo en el Sistema Electrónico de Contratación Pública SECOP I.

- **Tablas TVEC**: Contienen la información de la tienda virtual del estado colombiano.

El equipo EMAE procesará los datos desde el datalake de desarrollo, particularmente tomando los archivos en las carpetas bronze y posteriormente dejará todos sus resultados en las carpetas silver y gold. 

![image.png](../../../muestras%20de%20datos/sin%20procesar/Ingesta%20Bodega%20a%20DL.png)

## 4. ADQUISICIÓN DE LOS DATOS

### 4.1. Datos de procesos SECOP II

Para consultar los datos de procesos SECOP II, se debe ejecutar la siguiente consulta:

```sql
SELECT * FROM [CCE_Sandbox].[datavault].[SAT_Example_Table]
```


In [ ]:
# # Consulta de contratos depurados
# n=pd.read_sql_query("SELECT COUNT(*) FROM [CCE_Reports].[SECOPII].[V_HistoricoContratos_Depurado]", cnxn).values[0][0]
# Lista_Contratos_Depurado=[]
# for i in range(0,n,1000000):
#     print("Inicia la consulta en", i)
#     SECOP_II_Contratos_Depurado_temp = pd.read_sql_query("SELECT * FROM [CCE_Reports].[SECOPII].[V_HistoricoContratos_Depurado] ORDER BY [Nombre Entidad] OFFSET "+str(i)+" ROWS FETCH NEXT 1000000 ROWS ONLY", cnxn)
#     Lista_Contratos_Depurado.append(SECOP_II_Contratos_Depurado_temp)

# SECOP_II_Contratos_Depurado=pd.concat(Lista_Contratos_Depurado)



# ## Ruta de secop II contratos depurados
# ruta_Large="/dropbox/Dropbox/CCE-projects/Large Data/"
# SECOP_II_Contratos_Depurado.to_parquet(ruta_Large+"SECOP_II_Contratos_Depurado.parquet", engine='pyarrow')


# ## Consulta de SECOP II procedimientos depurados
# n=pd.read_sql_query("SELECT COUNT(*) FROM [CCE_Reports].[SECOPII].[V_HistoricoProcedimientos_Depurado]", cnxn).values[0][0]
# Lista_Procesos_Depurado=[]
# for i in range(0,n,100000):
#     print("Inicia la consulta en", i)
#     SECOP_Procesos_Depurado_temp = pd.read_sql_query("SELECT * FROM [CCE_Reports].[SECOPII].[V_HistoricoProcedimientos_Depurado] ORDER BY [Nombre Entidad] OFFSET "+str(i)+" ROWS FETCH NEXT 100000 ROWS ONLY", cnxn)
#     Lista_Procesos_Depurado.append(SECOP_Procesos_Depurado_temp)

# SECOP_II_Procesos_Depurado=pd.concat(Lista_Procesos_Depurado)
# SECOP_II_Procesos_Depurado = pd.read_sql_query("SELECT * FROM [CCE_Reports].[SECOPII].[V_HistoricoProcedimientos_Depurado]", cnxn)


# ## Se exportan a un parquet los procesos depurados de SECOP II
# SECOP_II_Procesos_Depurado.to_parquet(ruta_Large+"SECOP_II_Procesos_Depurado.parquet",engine='pyarrow')

# #
# ### cerramos la conexión
# cnxn.close()

In [ ]:
SECOP_II_Contratos = pd.read_sql_query("SELECT * FROM [CCE_Reports].[SECOPII].[HistoricoContratos]", cnxn_reports)
SECOP_II_Contratos_Depurado = pd.read_sql_query("SELECT  * FROM [CCE_Reports].[SECOPII].[V_HistoricoContratos_Depurado]", cnxn_reports)
SECOP_II_Procesos = pd.read_sql_query("SELECT  * FROM [CCE_Reports].[SECOPII].[HistoricoProcedimientos_Despliegue]", cnxn_reports)
SECOP_I_Procesos = pd.read_sql_query("SELECT * FROM [CCE_Sandbox].[SECOPI].[V_SECOPI]", cnxn_sandbox)
SECOP_I_Contratos = pd.read_sql_query("SELECT  * FROM [CCE_Sandbox].[SECOPI].[VWAdjudicacionesDepuradas]", cnxn_sandbox)

## cnxn es la conexión a la base de datos que se estableció en celdas anteriores

In [ ]:
## ruta_Large es la ruta donde se guardará el archivo parquet

ruta_Large="/dropbox/Dropbox/CCE-projects/Data/"
SECOP_II_Contratos.to_parquet(ruta_Large+"SECOP_II_Contratos.parquet", engine='pyarrow')
SECOP_II_Contratos_Depurado.to_parquet(ruta_Large+"SECOP_II_Contratos_Depurado.parquet",engine='pyarrow')
SECOP_II_Procesos.to_parquet(ruta_Large+"SECOP_II_Procesos.parquet",engine='pyarrow')
SECOP_I_Procesos.to_parquet(ruta_Large+"SECOP_I_Procesos.parquet",engine='pyarrow')
SECOP_I_Contratos.to_parquet(ruta_Large+"SECOP_I_Contratos.parquet",engine='pyarrow')

### cerramos la conexión
cnxn_reports.close()
cnxn_sandbox.close()

## Guardado en DataLake

Usamos un token SAS (Shared Access Signature) para acceder al DataLake. Este token se genera en el portal de Azure, en la cuenta de almacenamiento, en la sección de "Shared access signature". El token se genera con permisos de lectura y escritura, y con una duración de 3 meses.

Un token SAS es una cadena de caracteres que incluye una firma criptográfica generada por el servidor y que contiene información sobre los permisos y la duración del acceso autorizado. Al generar un token SAS, puedes especificar qué acciones se permiten (por ejemplo, leer, escribir, eliminar), qué recursos se pueden acceder y durante cuánto tiempo el token será válido.

El uso de tokens SAS te permite delegar el acceso a los recursos de Azure Storage sin compartir tus claves de acceso privadas. Esto proporciona un nivel adicional de seguridad y control sobre tus datos almacenados en Azure.

Para generar un token SAS, puedes utilizar las bibliotecas de desarrollo de Azure Storage, la interfaz de línea de comandos de Azure (Azure CLI) o el Portal de Azure. Una vez generado el token SAS, puedes compartirlo con terceros para que puedan acceder a los recursos de Azure Storage durante el período y con los permisos especificados.

Es importante tener en cuenta que los tokens SAS son válidos solo durante el período de tiempo especificado y tienen las restricciones y permisos definidos al generarse. Por lo tanto, es fundamental configurar adecuadamente los permisos y la duración del token SAS para garantizar la seguridad y el acceso adecuado a tus datos almacenados en Azure Storage.

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

token_sas=os.getenv("token_sas")
url_blob=os.getenv("url_blob")

### Conectamos con el DataLake
# Conexión a la cuenta de almacenamiento de Azure utilizando el token SAS
connect_str = f"BlobEndpoint={url_blob};SharedAccessSignature={token_sas}"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [ ]:
# Referencia al contenedor donde se encuentran los archivos
container_name_ent = "datosemae/bronze/SECOP_ReportsVersion"
container_client_ent = blob_service_client.get_container_client(container_name_ent)

# Nombre del archivo a leer
blob_name_ent_secop_ii_con = "SECOP_II_Contratos.parquet"
blob_name_ent_secop_ii_con_dep = "SECOP_II_Contratos_Depurado.parquet"
blob_name_ent_secop_ii_pro = "SECOP_II_Procesos.parquet"
blob_name_ent_secop_i_con = "SECOP_I_Contratos.parquet"
blob_name_ent_secop_i_pro = "SECOP_I_Procesos.parquet"


## Carga de datos usando Python desde la ruta DATA

Cargamos la base desde la ruta DATA




In [ ]:
import pandas as pd
ruta_Large="/dropbox/Dropbox/CCE-projects/Data"

In [ ]:
# Cargamos la ruta de los archivos
ruta_ent_secop_ii_con = ruta_Large+"/SECOP_II_Contratos.parquet"
ruta_ent_secop_ii_con_dep = ruta_Large+"/SECOP_II_Contratos_Depurado.parquet"
ruta_ent_secop_ii_pro = ruta_Large+"/SECOP_II_Procesos.parquet"
ruta_ent_secop_i_con = ruta_Large+"/SECOP_I_Contratos.parquet"
ruta_ent_secop_i_pro = ruta_Large+"/SECOP_I_Procesos.parquet"


## Nos conectamos al cliente de Azure

blob_client_secop_ii_con = blob_service_client.get_blob_client(container=container_name_ent, blob=blob_name_ent_secop_ii_con)
blob_client_secop_ii_con_dep = blob_service_client.get_blob_client(container=container_name_ent, blob=blob_name_ent_secop_ii_con_dep)
blob_client_secop_ii_pro = blob_service_client.get_blob_client(container=container_name_ent, blob=blob_name_ent_secop_ii_pro)
blob_client_secop_i_con = blob_service_client.get_blob_client(container=container_name_ent, blob=blob_name_ent_secop_i_con)
blob_client_secop_i_pro = blob_service_client.get_blob_client(container=container_name_ent, blob=blob_name_ent_secop_i_pro)

### Cargamos los archivos al DataLake

with open(ruta_ent_secop_ii_con, "rb") as data:
    blob_client_secop_ii_con.upload_blob(data)

with open(ruta_ent_secop_ii_con_dep, "rb") as data:
    blob_client_secop_ii_con_dep.upload_blob(data)

with open(ruta_ent_secop_ii_pro, "rb") as data:
    blob_client_secop_ii_pro.upload_blob(data)

with open(ruta_ent_secop_i_con, "rb") as data:
    blob_client_secop_i_con.upload_blob(data)

with open(ruta_ent_secop_i_pro, "rb") as data:
    blob_client_secop_i_pro.upload_blob(data)

## Cierre

En este ejemplo se mostró como usando python se puede acceder a los datos de la bodega y cargarlos en el DataLake. Vale la pena resaltar que puede ser más provechoso usar Azure CLI para realizar estas tareas.